In [5]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

gpg: keybox '/tmp/tmpfxqpsqoc/pubring.gpg' created
gpg: /tmp/tmpfxqpsqoc/trustdb.gpg: trustdb created
gpg: key AD5F235DF639B041: public key "Launchpad PPA for Alessandro Strada" imported
gpg: Total number processed: 1
gpg:               imported: 1
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
!pip install -q keras

In [8]:
#check that you are using GPU
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [9]:
!ls

datalab  drive


In [0]:
import os
os.chdir("drive/Colab")

In [11]:
!ls

MNIST_NN_colab.ipynb


In [12]:
!ls

MNIST_NN_colab.ipynb  test_MNIST.csv  train_MNIST.csv


In [14]:
!pwd

/content/drive/Colab


In [0]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import csv as csv
import matplotlib.pyplot as plt
import numpy as np

In [0]:
##manual download, then use pd
test_mnist= pd.read_csv('test_MNIST.csv', sep=',')
train_mnist = pd.read_csv('train_MNIST.csv', sep=',')

In [0]:
# split train - test and convert to numpy arrays with .values
X_train, y_train= train_mnist.iloc[:, 1:].values, train_mnist.iloc[:, 0].values

In [84]:
print(X_train.shape)
print(y_train.shape)

(42000, 784)
(42000,)


In [0]:
n_train = 42000

***Data augmentation***

see here: https://machinelearningmastery.com/image-augmentation-deep-learning-keras/



In [0]:
#Create and configure your ImageDataGenerator
datagen = ImageDataGenerator(rotation_range=8, width_shift_range=0.08, shear_range=0.3,height_shift_range=0.08, zoom_range=0.08)
#fit it on train data
datagen.fit(train)
X_batch, y_batch = datagen.flow(train, train, batch_size=32)
fit_generator(datagen, samples_per_epoch=len(train), epochs=100)


# Train keras model

In [0]:
from keras.utils.np_utils import to_categorical
# we have to preprocess the data into the right shape
X_train = X_train.reshape(n_train, 28, 28, 1).astype('float32')
# normalize from [0, 255] to [0, 1]
X_train /= 255
# numbers 0-9, so ten classes
n_classes = 10
# convert integer labels into one-hot vectors
y_train = to_categorical(y_train, n_classes)

In [0]:
from keras.models import Sequential
model = Sequential()

In [0]:
# number of convolutional filters
n_filters = 32
# convolution filter size
# i.e. we will use a n_conv x n_conv filter
n_conv = 3
# pooling window size
# i.e. we will use a n_pool x n_pool pooling window
n_pool = 2

In [0]:
from keras.layers import Activation
from keras.layers.convolutional import Convolution2D, MaxPooling2D

model.add(Convolution2D(n_filters, kernel_size=(n_conv, n_conv),
        # we have a 28x28 single channel (grayscale) image so the input shape should be (28, 28, 1)
        input_shape=(28, 28, 1)))

model.add(Activation('relu'))

model.add(Convolution2D(n_filters, kernel_size=(n_conv, n_conv)))
model.add(Activation('relu'))

# then we apply pooling to summarize the features
# extracted thus far
model.add(MaxPooling2D(pool_size=(n_pool, n_pool)))

In [0]:
from keras.layers import Dropout, Flatten, Dense

model.add(Dropout(0.25))

# flatten the data for the 1D layers
model.add(Flatten())

# Dense(n_outputs)
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))

# the softmax output layer gives us a probablity for each class
model.add(Dense(n_classes))
model.add(Activation('softmax'))

In [0]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [93]:
# how many examples to look at during each update step
batch_size = 1300

# how many times to run through the full set of examples
n_epochs = 50

# the training may be slow depending on your computer
model.fit(X_train, y_train, batch_size=batch_size, epochs=n_epochs)

Epoch 1/50
42000/42000 [==============================] - 4s 84us/step - loss: 0.0707 - acc: 0.9784
Epoch 2/50
42000/42000 [==============================] - 3s 70us/step - loss: 0.0639 - acc: 0.9802
Epoch 3/50
42000/42000 [==============================] - 3s 69us/step - loss: 0.0546 - acc: 0.9839
Epoch 4/50
42000/42000 [==============================] - 3s 69us/step - loss: 0.0514 - acc: 0.9839
Epoch 5/50
42000/42000 [==============================] - 3s 70us/step - loss: 0.0478 - acc: 0.9845
Epoch 6/50
42000/42000 [==============================] - 3s 71us/step - loss: 0.0433 - acc: 0.9865
Epoch 7/50
42000/42000 [==============================] - 3s 70us/step - loss: 0.0413 - acc: 0.9877
Epoch 8/50
42000/42000 [==============================] - 3s 70us/step - loss: 0.0380 - acc: 0.9882
Epoch 9/50
42000/42000 [==============================] - 3s 70us/step - loss: 0.0365 - acc: 0.9879
Epoch 10/50
42000/42000 [==============================] - 3s 70us/step - loss: 0.0333 - acc: 0.9895

In [49]:
X_test = test_mnist.values.astype('float32')
X_test

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [50]:
print(X_test.shape)

(28000, 784)


In [0]:
n_test= 28000

In [0]:
#reshape X_test as I did for X_train before
X_test = X_test.reshape(n_test, 28, 28, 1)
# normalize from [0, 255] to [0, 1]
X_test /= 255

In [96]:
#now finally both test and train have the same dimension
print(X_test.shape)
print(X_train.shape)

(28000, 28, 28, 1)
(42000, 28, 28, 1)


In [98]:
# prediction
yPred = model.predict_classes(X_test,verbose=1)
#convert to integer
yPred = yPred.astype(int)
#create a series with the predictions
results = pd.Series(yPred,name="Label")
# then a second series with the index from 1 to x
index= pd.Series(range(1,28001), name = "ImageId")
#join the two series into a single dataframe
submission = pd.concat([index, results], axis = 1)
#save results to csv
submission.to_csv("CNN_mnist3.csv",index=False)

28000/28000 [==============================] - 3s 103us/step


other usefu resource
https://yashk2810.github.io/Applying-Convolutional-Neural-Network-on-the-MNIST-dataset/